Mpunt google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import data clinical sample and data mutations file

In [2]:
import pandas as pd

data_clinical = pd.read_csv('/content/drive/MyDrive/Dr. V/data_clinical_sample.txt', sep='\t', skiprows = 4, header = 0)
data_mutation = pd.read_csv('/content/drive/MyDrive/Dr. V/data_mutations.txt', sep='\t', header = 0)
#print (data_clinical.head())

Filtering required columns from data_clinical

In [8]:

select_col_filtered = ['SAMPLE_ID', 'PATIENT_ID','CANCER_TYPE','CANCER_TYPE_DETAILED']

clinical = data_clinical[select_col_filtered]
print(clinical.columns)

Index(['SAMPLE_ID', 'PATIENT_ID', 'CANCER_TYPE', 'CANCER_TYPE_DETAILED'], dtype='object')


Filtering columns in the data mutation file

In [13]:
select_col_mut = ['Hugo_Symbol','Consequence','Reference_Allele',
              'Tumor_Seq_Allele1','Tumor_Seq_Allele2','Tumor_Sample_Barcode','HGVSp']
mutation = data_mutation[select_col_mut]
print(mutation.shape)

(208953, 7)


Types of cancers in MSK CHORD dataset

In [9]:
cancer_type = clinical['CANCER_TYPE'].unique()
print(cancer_type)

['Non-Small Cell Lung Cancer' 'Breast Cancer' 'Colorectal Cancer'
 'Pancreatic Cancer' 'Prostate Cancer']


For loop to generate cindividual cancer mutation files

In [15]:
cancer_files = {}
for cancer in cancer_type:
  filtered_df = clinical[clinical['CANCER_TYPE'] == cancer]
  # extracting sample id
  sample_id = filtered_df['SAMPLE_ID']
  # match the sample id with the mutation sample barcode
  matched_mutation_df = mutation[mutation['Tumor_Sample_Barcode'].isin(sample_id)]
  #merge the mutation and clinical data using the matched id's
  merged_df = pd.merge(filtered_df, matched_mutation_df, left_on='SAMPLE_ID', right_on='Tumor_Sample_Barcode', how = 'inner')
  # store the merged data in the dictionary
  cancer_files[cancer] = merged_df
  #write the merged_df for each cancer as a .txt file
  merged_df.to_csv(f"/content/drive/MyDrive/Dr. V/{cancer}_mutation_data.txt", sep='\t', index=False)
